In [ ]:
# Importando bibliotecas
import pprint
import pandas as pd
import copy
import matplotlib.pyplot as plt
import os

# Importando arquivos
from cycles import calculate_basic_cycle, calculate_two_evaporators_cycle
from optimization import optimize_two_evaporators_cycle_with_multiple_refrigerants, optimize_basic_cycle_with_multiple_refrigerants

In [ ]:
# Temperatura externa: https://en.climate-data.org/south-america/brazil/rio-de-janeiro/rio-de-janeiro-853/ (media das maximas)
# Temperaturas internas: pagina 18 do arquivo MFL70720817_PB.pdf (LG)
# Refrigerante base: pagina 6 do arquivo MFL70720817_PB.pdf (LG)
# Consumo: pagina 16 do arquivo CATALOGO_10_BRASTEMP (Brastemp)
# Tarifa media do kWh: https://www.aneel.gov.br/ranking-das-tarifas (Light RJ)

# Temperaturas novas por mes: https://www.climatempo.com.br/climatologia/321/riodejaneiro-rj
default_input_values = {
    't_internal_env_ht': 3 + 273.15,
    't_internal_env_lt': -18 + 273.15,
    'approach_condenser': 5,
    'approach_evaporator_ht': 5,
    'approach_evaporator_lt': 5,
    'q_evaporator_ht': 75,
    'q_evaporator_lt': 75,
    'isentropic_efficiency': 0.7,
    'subcooling': 0,
    'superheating_ht': 0,
    'superheating_lt': 0,
}

input_values = {
    't_internal_env_ht': 3 + 273.15,
    't_internal_env_lt': -18 + 273.15,
    'approach_condenser': 5,
    'approach_evaporator_ht': 5,
    'approach_evaporator_lt': 5,
    'f': 0.5,
    'isentropic_efficiency': 0.7,
    'subcooling': 5,
    'superheating_ht': 5,
    'superheating_lt': 5,
    'upper_threshold': 10,
    'lower_threshold': 0
}

input_ranges = {
    'refrigerants': ['R600a', 'R134a', 'R1234yf', 'R1234ze(E)', 'R22', 'R290', 'NH3', 'R404a', 'R410a'],
    't_external_env_month': [['Janeiro', 29], ['Fevereiro', 30], ['Março', 29], ['Abril', 28],
                             ['Maio', 26], ['Junho', 25], ['Julho', 24], ['Agosto', 25],
                             ['Setembro', 25], ['Outubro', 26], ['Novembro', 27], ['Dezembro', 28]]
}

optimized_cop_table = optimize_two_evaporators_cycle_with_multiple_refrigerants(default_input_values, 
                                                                                input_values, 
                                                                                'cop', 
                                                                                input_ranges)
optimized_cop_table.to_excel(os.getcwd() + r'\results\two_evaporators_cop.xlsx', index = False)

optimized_exergy_efficiency_table = optimize_two_evaporators_cycle_with_multiple_refrigerants(default_input_values, 
                                                                                              input_values, 
                                                                                              'exergy_efficiency_components', 
                                                                                              input_ranges)
optimized_exergy_efficiency_table.to_excel(os.getcwd() + r'\results\two_evaporators_exergy_efficiency.xlsx', index = False)

In [ ]:
default_input_values = {
    't_internal_env': 21 + 273.15,
    'approach_condenser': 5,
    'approach_evaporator': 5,
    'q_evaporator': 9000 * 0.293071,
    'isentropic_efficiency': 0.7,
    'subcooling': 0,
    'superheating': 0,
}

input_values = {
    't_internal_env': 21 + 273.15,
    'approach_condenser': 5,
    'approach_evaporator': 5,
    'isentropic_efficiency': 0.7,
    'subcooling': 5,
    'superheating': 5,
    'upper_threshold': 10,
    'lower_threshold': 0
}

input_ranges = {
    'refrigerants': ['R600a', 'R134a', 'R1234yf', 'R1234ze(E)', 'R22', 'R290', 'NH3', 'R404a', 'R410a'],
    't_external_env_month': [['Janeiro', 29], ['Fevereiro', 30], ['Março', 29], ['Abril', 28],
                             ['Maio', 26], ['Outubro', 26], ['Novembro', 27], ['Dezembro', 28]]
}

optimized_cop_table = optimize_basic_cycle_with_multiple_refrigerants(default_input_values, 
                                                                      input_values,
                                                                      'cop',
                                                                      input_ranges)
optimized_cop_table.to_excel(os.getcwd() + r'\results\basic_cop.xlsx', index = False)

optimized_exergy_efficiency_table = optimize_basic_cycle_with_multiple_refrigerants(default_input_values, 
                                                                                    input_values,
                                                                                    'exergy_efficiency_components',
                                                                                    input_ranges)
optimized_exergy_efficiency_table.to_excel(os.getcwd() + r'\results\basic_exergy_efficiency.xlsx', index = False)

In [ ]:
# Usando as variaveis de entrada do Yang (2015).pdf para comparar os ciclos
input_values = {
    't_external_env': 30 + 273.15,
    't_internal_env_ht': -5 + 273.15,
    't_internal_env_lt': -26 + 273.15,
    'approach_condenser': 0,
    'approach_evaporator_ht': 0,
    'approach_evaporator_lt': 0,
    'q_evaporator_ht': 239,
    'q_evaporator_lt': 246,
    'isentropic_efficiency': 0.78,
    'superheating_ht': 0,
    'superheating_lt': 0,
    'refrigerant': 'R600a'
}

input_ranges = {
    'subcooling': range(0,21)
}

original_input_values = copy.copy(input_values)
results = pd.DataFrame(columns=['cop',
                                'subcooling'])
for subcooling in input_ranges['subcooling']:
    input_values = copy.copy(original_input_values)
    input_values['subcooling'] = subcooling
    cycle = calculate_two_evaporators_cycle(input_values)
    results = results.append({
        'cop': cycle['cop'],
        'subcooling': subcooling
    }, ignore_index=True)

results.plot(x='subcooling', y='cop', kind='scatter', xticks=range(0,21), yticks=[2 + (x * 0.1) for x in range(6, 13)])
pprint.pprint(results)